<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# LSTM para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# Objetivo

Diseñar una red LSTM para predecir el comportamiento del índice S&P 500 utilizando datos reales de Yahoo Finance.

Además de construir el modelo, analizaremos el **efecto de los principales hiperparámetros** de la capa LSTM en Keras. Considerando:


1. Descargar y preparar datos financieros reales.  
2. Construir una red LSTM paso a paso.  
3. Comprender el papel de los hiperparámetros:  
    `units`, `return_sequences`, `dropout`, `recurrent_dropout`, `batch_size`, `epochs`, `optimizer`.  
4. Visualizar cómo cambian los resultados al modificarlos.

In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [5]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

In [3]:
# Descargar datos del S&P 500
data = yf.download('^GSPC', start='2018-01-01', end='2025-01-01')

# Aplanar columnas en caso de que vengan con MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [' '.join(col).strip() for col in data.columns.values]

# Verificamos si existe la columna 'Close'
if 'Close' in data.columns:
    close_series = data['Close']
else:
    # En algunos casos se llama 'Close ^GSPC'
    col_close = [c for c in data.columns if 'Close' in c][0]
    close_series = data[col_close]

# Asegurar formato 1D numérico
close_series = pd.Series(close_series.values, index=pd.to_datetime(data.index))
close_series = pd.to_numeric(close_series, errors='coerce').dropna()

# Gráfico interactivo con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=close_series.index,
    y=close_series.values,
    mode='lines',
    line=dict(color='royalblue'),
    name='Cierre S&P 500'
))

fig.update_layout(
    title='S&P 500 — Precio de Cierre Diario (2018–2024)',
    xaxis_title='Fecha',
    yaxis_title='Precio (USD)',
    hovermode='x unified',
    yaxis=dict(tickprefix='$', separatethousands=True)
)

fig.show()


/var/folders/fd/q5kbvbvd00b4rdf_lfzx0j3m0000gp/T/ipykernel_11967/694743252.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('^GSPC', start='2018-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


In [ ]:
# # --- CHUNK: descarga y guardado del dataset ---

# import os
# import pandas as pd
# import yfinance as yf

# # 1. Crear carpeta donde se guardarán los datos
# data_dir = "data_raw"
# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

# # 2. Descargar datos (ejemplo: MSFT)
# symbol = "MSFT"
# from_date = "2015-01-01"
# to_date   = "2020-12-31"

# data = yf.download(symbol, start=from_date, end=to_date)

# # 3. Guardar el dataset como CSV dentro de la carpeta
# csv_path = os.path.join(data_dir, f"{symbol.lower()}_raw.csv")
# data.to_csv(csv_path)

# print(f"Dataset guardado en: {csv_path}")

# # 4. Leer nuevamente desde el archivo guardado
# df_loaded = pd.read_csv(csv_path)

# df_loaded.head()


/var/folders/fd/q5kbvbvd00b4rdf_lfzx0j3m0000gp/T/ipykernel_11967/300469600.py:17: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Dataset guardado en: data_raw/msft_raw.csv


,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2015-01-02,39.93304443359375,40.49668438609426,39.745166621261546,39.84764572414747,27913900
3,2015-01-05,39.565834045410156,39.907432198522,39.49751246013629,39.5999915802946,39673900
4,2015-01-06,38.98512649536133,39.92452579930231,38.891185913416514,39.60854650728631,36447900


1. Escalamiento de los datos.
Cual escalador podemos utilizar?

2. Definir entrada de datos. Como se pueden ingresar los datos a un LSTM?

3. Como se declara un modelos LSTM en Keras?